In [41]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from datetime import datetime

In [42]:
# zipmap.net/California/Santa_Clara_County.html
# sunnyvale - 94085, 94086, 94087, 94089
# cupertino - 95014
# los altos - 94024, 94022
# saratoga - 95070
# redwood city - 94063, 94061
## added on 5/18
# santa clara - 95050, 95051, 95128
# west san jose - 95129
# campell - 95117


## Could be added later
# mountain view - 94040, 94041, 94043
# west san jose - 95129
# campell - 95130, 95008

In [43]:
zipcodes = [94087, 94086, 94085, 94089, 95014, 94024, 94022, 95070, 94063,94061, 95050, 95051, 95128, 95129, 95117]

In [44]:
#https://free-proxy-list.net/
proxy_list = ["37.120.192.154:8080", "201.148.70.100:80", "118.69.50.154:80",
              "203.124.39.150:80", "104.154.143.77:3128", "143.0.92.162:80",
              "164.163.73.66:999", "138.197.157.32:8080", "42.3.51.114:80",
              "103.105.49.53:80", "52.149.71.249:80", "115.249.2.192:8080",
              "176.9.75.42:8080", "203.202.245.62:80", "139.99.135.214:80",
              "138.68.41.90:3128", "88.198.50.103:8080", "46.4.96.137;3128",
              "78.93.58.60:3128", "191.233.199.87:80", "138.68.161.14:3128"]

In [45]:
df = pd.DataFrame(columns = ['zipcode','address', 'listing_type', 'price', 'num_bds', 'num_ba', 'sqft_num', 'details', 'url'])
proxy_index = 0
for zc in tqdm(zipcodes):
    list_card = []
    while len(list_card) == 0:
        proxy_use = proxy_list[proxy_index]
        print(proxy_use)
        proxy_index += 1
        url = f'https://www.zillow.com/homes/for_sale/{zc}/0_singlestory/days_sort'
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument(f'--proxy-server=http://{proxy_use}')
        driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=chrome_options)
        driver.get(url)
        time.sleep(30)
        html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        soup = BeautifulSoup(html, 'html.parser')
        driver.close()

        zipcode = zc
        list_card = soup.find_all(class_ = "list-card-info")
    proxy_index -= 1
    
    for i in list_card:
        url = i.find('a')['href']
        address = i.find(class_ = "list-card-addr").text
        listing_type = i.find(class_ = "list-card-type").text
        price = i.find(class_ = "list-card-price").text
        details = i.find(class_ = "list-card-details").text
    
        if 'bds' in details: 
            bds_loc = details.find('bds')
            ba_loc = details.find('ba')
            sqft_loc = details.find('sqft')

            num_bds = details[0:bds_loc].strip()
            num_ba = details[bds_loc+3:ba_loc].strip()
            sqft_num = details[ba_loc+2:sqft_loc].strip()
        elif 'bd' in details: 
            bds_loc = details.find('bd')
            ba_loc = details.find('ba')
            sqft_loc = details.find('sqft')

            num_bds = details[0:bds_loc].strip()
            num_ba = details[bds_loc+2:ba_loc].strip()
            sqft_num = details[ba_loc+2:sqft_loc].strip()
        else:
            num_bds = ''
            num_ba = ''
            sqft_num = ''

        row = [zipcode, address, listing_type, price, num_bds, num_ba, sqft_num, details, url]
        df.loc[len(df)] = row
    time.sleep(30)
    
    
df['extract_ts'] = str(datetime.now())

37.120.192.154:8080


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


201.148.70.100:80
118.69.50.154:80
203.124.39.150:80
104.154.143.77:3128
143.0.92.162:80
164.163.73.66:999
138.197.157.32:8080
42.3.51.114:80
103.105.49.53:80
52.149.71.249:80
115.249.2.192:8080
176.9.75.42:8080
203.202.245.62:80
139.99.135.214:80
138.68.41.90:3128
88.198.50.103:8080
46.4.96.137;3128
78.93.58.60:3128
191.233.199.87:80
138.68.161.14:3128


IndexError: list index out of range

In [40]:
df.shape

(303, 9)

In [31]:
df.groupby(['zipcode']).nunique()

,zipcode,address,listing_type,price,num_bds,num_ba,sqft_num,details,url,extract_ts
zipcode,,,,,,,,,,
94022,1,40,8,39,7,12,35,38,40,1
94024,1,29,3,25,7,11,28,29,29,1
94061,1,40,5,39,7,6,33,40,40,1
94063,1,37,10,37,6,7,33,35,37,1
94085,1,30,6,29,5,5,22,24,40,1
94086,1,40,8,37,6,6,39,39,40,1
94087,1,30,6,28,5,6,28,29,30,1
94089,1,40,6,36,6,5,32,33,40,1
95014,1,40,5,40,6,6,36,38,40,1


In [32]:
df.head()

,zipcode,address,listing_type,price,num_bds,num_ba,sqft_num,details,url,extract_ts
0,94087,"709 Satsuma Ter, Sunnyvale, CA 94087",Townhouse for sale,"$1,598,000",3,3,"1,851","3 bds3 ba1,851 sqft",https://www.zillow.com/homedetails/709-Satsuma...,2020-06-23 01:05:11.581914
1,94087,"855 Mango Ave, Sunnyvale, CA 94087",House for sale,"$1,798,000",3,2,"1,476","3 bds2 ba1,476 sqft",https://www.zillow.com/homedetails/855-Mango-A...,2020-06-23 01:05:11.581914
2,94087,"1105 Heatherstone Way, Sunnyvale, CA 94087",House for sale,"$1,598,000",3,1,"1,130","3 bds1 ba1,130 sqft",https://www.zillow.com/homedetails/1105-Heathe...,2020-06-23 01:05:11.581914
3,94087,"1306 Cordilleras Ave, Sunnyvale, CA 94087",House for sale,"$1,595,888",3,1,"1,034","3 bds1 ba1,034 sqft",https://www.zillow.com/homedetails/1306-Cordil...,2020-06-23 01:05:11.581914
4,94087,"787 W Homestead Rd, Sunnyvale, CA 94087",House for sale,"$1,998,950",4,2,"1,919","4 bds2 ba1,919 sqft",https://www.zillow.com/homedetails/787-W-Homes...,2020-06-23 01:05:11.581914


In [33]:
df.to_csv('data/zillow_2020-06-22.csv')

In [52]:
for zc in tqdm(zipcodes):
    url = f'https://www.zillow.com/homes/for_sale/{zc}/0_singlestory/days_sort'
    driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=chrome_options)
    driver.get(url)
    time.sleep(30)
    html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    
    listing_url = []
    for listing in soup.find(id='grid-search-results').find_all('article'):
        listing_url.append(listing.find('a')['href'])
       
    for l in tqdm(listing_url):
        driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=chrome_options)
        driver.get(l)
        time.sleep(30)
        html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        soup = BeautifulSoup(html, 'html.parser')
        driver.close()
        price = soup.find(class_="ds-value").text
        
        home_stats = []
        for i in soup.find_all(class_="ds-bed-bath-living-area"):
            home_stats.append(i.text)
        home_stats = remove_dup(home_stats)
        
        num_bedroom = home_stats[0]
        num_bathroom = home_stats[1]
        sqft = home_stats[2]

        address = soup.find(class_="ds-address-container").text.replace('\xa0', '')
        status = soup.find(class_="ds-status-details").text

        feature_dict = {}
        for i in soup.find_all(class_="ds-home-fact-list-item"):
            key = i.text.split(':')[0]
            feature_dict[key] = i.text.split(':')[1]

        home_type = feature_dict['Type']
        year_built = feature_dict['Year built']
        parking = feature_dict['Parking']
        price_sqft = feature_dict['Price/sqft']
        
        row = [address, status, price, num_bedroom, num_bathroom, sqft,
               home_type, year_built, parking, price_sqft, url]
        
        df.loc[len(df)] = row
        time.sleep(120)
    time.sleep(300)
    
df['extract_ts'] = str(datetime.now())
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=81.0.4044.113)
